In [17]:
from RunGreatCircle import great_circle

In [18]:
print(great_circle(-13,73,-105,38))

3762.884718393407


In [19]:
import numpy as np
import time
n = 1000000
m = np.random.randint(-360,360,n*4).reshape(n,4)

In [20]:
import subprocess


# run python
timeStart = time.time()
cmd = 'python RunGreatCircle.py'
proc = subprocess.call(cmd,shell=True)
runTime = time.time() - timeStart
print("Python time", time.strftime('%H:%M:%S', time.gmtime(runTime)))

Python time 00:00:03


In [21]:
import subprocess


# run python
timeStart = time.time()
cmd = 'python RunGreatCircle.py'
proc = subprocess.call(cmd,shell=True)
runTime = time.time() - timeStart
print("Python time", time.strftime('%H:%M:%S', time.gmtime(runTime)))

Python time 00:00:00


In [25]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import os,subprocess,threading

def run_subprocess(cmd):
    proc = subprocess.Popen(cmd,shell=True,stdout=subprocess.PIPE,stdin=subprocess.PIPE)
    while True:
        try:
            next_line = proc.stdout.readline()
            proc.wait()
            if next_line == '' and proc.poll() != None:
                break
        except:
            proc.wait()
            break

class RunSubprocess(object):
    """
    a generic class to control a subprocess with threads
    """

    def __init__(self, cmd, mainWindow=None):
        self.cmd = cmd
        self.mainWindow = mainWindow
        self.process = None
        self.stdOut,self.stdErr = None,None

    def run(self,timeout=100):
        def target():
            self.process = subprocess.Popen(self.cmd,shell=True,stderr=subprocess.PIPE,
                                stdout=subprocess.PIPE,universal_newlines=True,bufsize=4096)

            self.stdOut, self.stdErr = self.process.communicate()

        self.thread = threading.Thread(target=target)
        self.thread.start()

        ## wait a specified amount of time before terminating
        if timeout != None:
            self.thread.join(timeout)
            if self.thread.is_alive():
                print('The subprocess was auto-terminated due to timeout')
                print("...", self.process.poll())
                self.process.terminate()
                self.thread.join()
        
            return self.process.returncode
        return None

    def add_text(self):
        if self.textScreen == None:
            return
        
        if self.textScreen.showMessages == False:
            self.textScreen.toggle_message_btn()
            
        self.textScreen.add_text(txt)
        QtGui.QApplication.processEvents()

    def terminate(self):
        if self.thread.is_alive():
            self.process.terminate()
            self.thread.join()

if __name__ == '__main__':
    
    myProcess = RunSubprocess("echo 'Process started'; sleep 2; echo 'Process finished'")
    
    ## test should pass
    returnCode = myProcess.run(timeout=10)
    print('pass return code', returnCode)

    ## test should fail
    returnCode = myProcess.run(timeout=1)
    print('fail return code', returnCode)


pass return code 0
The subprocess was auto-terminated due to timeout
... None
fail return code -15


# Multiprocessing

In [ ]:
from multiprocessing import Pool, cpu_count
total_cores = cpu_count()
print('total cores: ', total_cores)

In [ ]:
import numpy as np
from multiprocessing import pool
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor

def f(x):
    return x*x

with ProcessPoolExecutor(max_workers=4) as pool:
    result = pool.map(f, range(10))
    result = np.array(list(result))
    print(result)

In [31]:
import time,math,os
from multiprocessing import Pool, cpu_count
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
import numpy as np


def func_timer(func):
    """Times how long the function took."""

    def f(*args, **kwargs):
        import time
        start = time.time()
        results = func(*args, **kwargs)
        print("Elapsed: %.2fs" % (time.time() - start))
        return(results)
    return(f)
    
@func_timer
def test_function(msg, sleep=1.0):
    """Delays a while before answering."""
    import time
    time.sleep(sleep)
    print(msg)

@func_timer
def loop1(n):
    """Using for loop with function call."""
    z = []
    for i in range(n):
        z.append(math.sin(i))
    return z

@func_timer
def loop2(n):
    """Using local version of function."""
    z = []
    sin = math.sin
    for i in range(n):
        z.append(sin(i))
    return z

@func_timer
def loop3(n):
    """Using list comprehension."""
    sin = math.sin
    return [sin(i) for i in range(n)]

@func_timer
def loop4(n):
    """Using map."""
    sin = math.sin
    return list(map(sin, range(n)))

@func_timer
def loop5(n):
    """Using numpy."""
    return np.sin(np.arange(n)).tolist()


def calculate_great_circle(lon1,lat1,lon2,lat2):
    """one step of the great circle calculation"""
    lon1,lat1,lon2,lat2 
    
    radius = 3956.0
    x = np.pi/180.0
    a,b = (90.0-lat1)*(x),(90.0-lat2)*(x)
    theta = (lon2-lon1)*(x)

    c =  np.arccos((np.cos(a)*np.cos(b)) +
                   (np.sin(a)*np.sin(b)*np.cos(theta)))
    return(radius*c) 

@func_timer
def great_circle_looping(mat):
    """basic great circle"""

    result = np.zeros(mat.shape[0])
    for i in range(mat.shape[0]):
        result[i] = calculate_great_circle(mat[i,:])
        
    return result

@func_timer
def great_circle_numpy(mat):
    """numpys great circle"""
    radius = 3956
    x = np.pi/180.0
    lon1 = mat[:,0]
    lat1 = mat[:,1]
    lon2 = mat[:,2]
    lat2 = mat[:,3]
    
    a = (90.0-lat1)*(x)
    b = (90.0-lat2)*(x)
    theta = (lon2-lon1)*(x)
    c = np.arccos((np.cos(a)*np.cos(b)) +
                  (np.sin(a)*np.sin(b)*np.cos(theta)))
    return radius*c

class GreatCircleMultiprocessing(object):
    """multiprocessing version using async pool"""
    def __init__(self,mat):
        self.mat = mat
    
    def run(self,mat):
        po = Pool(processes=cpu_count()-1)
        _results = po.map_async(great_circle,(mat[i,:] for i in range(mat.shape[0])))
        results =  _results.get()
        return(results)

    def great_circle_multiprocessing(self,mat,chunksize=500):
        """multiprocessing version of the great circle usng Ececutors"""
        with ProcessPoolExecutor(max_workers=cpu_count()-1) as pool:
            result = pool.map(great_circle,(mat[i,:] for i in range(mat.shape[0])),chunksize=chunksize)

        return(np.array(list(result)))

'''
class GreatCircleMultiprocessing(object):
    def run(self,mat,chunksize=500):
        """multiprocessing version of the great circle usng Ececutors"""
        with ProcessPoolExecutor(max_workers=cpu_count()-1) as pool:
            result = pool.map(great_circle,(mat[i,:] for i in range(mat.shape[0])),chunksize=chunksize)

        return(np.array(list(result)))
'''
    
def great_circle_cython(mat):
    os.system("python setup_gc.py build_ext --inplace")
    from greatcircle import great_circle
    result = np.zeros(mat.shape[0])
    for i in range(mat.shape[0]):
        result[i] = great_circle(*mat[i,:])
    return(result)


if __name__ == "__main__":
    
    n = 500000
    m = np.random.randint(-360,360,n*4).reshape(n,4)

    @func_timer
    def run_slow_way(mat):
        for i in range(mat.shape[0]):
            x = great_circle(mat[i,:])

    ## slow way
    print("great circle via looping...")
    run_slow_way(m)
                   
    ## numpy way
    print("great circle via numpy...")
    great_circle_numpy(m)    
    



great circle via looping...


TypeError: great_circle() missing 3 required positional arguments: 'lat1', 'lon2', and 'lat2'